In [1]:
import sys
from pathlib import Path
import pandas as pd
sys.path.append('.')
from utils import remove_atom_mapping, canonicalize_smiles

data_dir = Path('../../data/uspto_50k')
output_dir = data_dir / 'positive'
splits = ['test', 'valid', 'train']

In [2]:
def load_raw_data(split: str):
    raw_path = data_dir / 'raw' / f'{split}.csv'
    raw = pd.read_csv(raw_path)
    column = raw.columns[-1]
    raw['reactants'] = raw[column].apply(lambda x: x.split('>>')[0]).apply(remove_atom_mapping)
    raw['product'] = raw[column].apply(lambda x: x.split('>>')[-1]).apply(remove_atom_mapping)
    raw['reactants'] = raw['reactants'].apply(canonicalize_smiles)
    raw['product'] = raw['product'].apply(canonicalize_smiles)
    raw = raw.drop(columns=[column, 'class', 'id'])
    raw['feasible'] = 1
    return raw

In [ ]:
output_dir.mkdir(exist_ok=True, parents=True)
for split in splits:
    raw = load_raw_data(split)
    raw.to_csv(output_dir / f'{split}.csv', index=False)